<span type="title">存储层次结构</span> | <span type="update">2018-07-17</span> | <span type="version">1</span>

<span type="intro"><p type="card-text">本章介绍SRAM和DRAM。</p></span>

# 存储层次结构概况

存储器的性能显而易见的会影响到计算机的性能，如果没有Cache，那么对于单周期处理器，其必须进行硬件停顿以等待数据写入（取指、执行、回写阶段），因此造成CPU周期成倍增加。

即便只有DRAM，其速度大约为CPU的1/100，那么性能损失接近100倍。更不用提流水线处理器，在这里，中断更为复杂。

![](w7p1.png)

如上图所示，CPU在30年来性能提升了2500倍，但是DRAM则仅仅为9倍，磁盘为29倍，SRAM提升幅度相比CPU亦然太小，200倍，不过对于其余部件来言，也是非常巨大的了。因此，当代计算机最大问题就是I/O阻塞的问题，而不是CPU性能问题。

实际上，从80386开始，已经有了片外的Cache，到了80486，Cache被集成到CPU内，从8KB到现在的32KB（不能增长过快的原因是流水线对速度的要求极高，容量必然带来速度的下降）。

# DRAM

## DRAM构成

![](w7p2.png)

对于内存而言，现在指的一般是DRAM，动态随机访问存储(D-R-A-M)。其构成如上图所示。对于PC，其内存模组主要有数个DRAM芯片组成，而对于手机等便携设备，DRAM直接焊接在SOC上。对于DRAM芯片，其包含行列选择器，数据输入输出口，缓存器和内存阵列，每个行列对应数个bit的原件，每个原件由一个电容和数条连线构成，每个cell可能包含不同bit的原件。

![](w7p3.png)

如上图所示的是一个bit的原件，其接受行列选择信号，当需要写入数据时，行列信号连通，数据输入1，经过刷新放大器，对电容C进行充电。当数据读出时，同样逆过程。

需要注意，对于DRAM，因为电容回漏电，因此需要一定时间进行刷新补充电荷。

## DRAM流程

![](w7p4.png)

如上所示，是DRAM的操作流程。CPU通过总线向内存控制器发送信号，内存控制器经过BUS通过Row选择器选择行，然后Memory Array将这一行所有数据信号都发送到Sense Amplifiers缓冲区，然后内存控制器再要求Column列选择器选择信号，其在缓冲区选择后经过Data In/oUT bUFFERS进行输出。

![](w7p5.png)

在这个过程中，从行选择到列选择之间因为要写入到缓冲区，所以有一个延时，称之为tRCD（PC133标准：2~3个时钟周期，约15~23ns）。

从列地址到读取数据的延迟，称之为CL（PC133标准：2~3个时钟周期，约15~23ns）。

之后读到数据的时候，每个数据块需要耗费1个时钟周期。因为不一定需要这一行，那么就要关闭这一行，称之为行预充电。

从行预充电（关闭行）到下一次行地址输入的延迟为tRP（PC133标准：2~3个时钟周期，约15~23ns）。

## DRAM发展

DRAM在1993年被改进为SDRAM,在2000年被改进为DDR SDRAM。其中SDRAM又称之为 SDR SDRAM，DDR SDRAM 称之为 DDR1 SDRAM。

DRAM的等效时钟频率为1600Mhz，意味着其一个周期需要0.625ns。其传输带宽为1600Mhz×64bit/8=12.8GB/s

大致上的计算公式为：`传输带宽 = 内存工作核心频率 * 位宽 * 倍增系数`

对于SDR，其倍增系数为1，对于DDR，其倍增系数为2，对于DDR2，其倍增系数为4，对于DDR3，其倍增系数为8。

内存工作核心频率指的是对于1byte而言，并且是对于内存而不是外部频率。比如`DDR3 1600: 1600/8=200Mhz（有效传输频率/8得到每部分的核心频率），200*64*8 = 102400Mbit，1024000/8 = 12800Mb/s = 12.8Gb/s`

影响SDRAM的关键因素在于电容充放电时间，这个因素是难以改进的。

对于 Double Data Rate SDRAM 而言，其实现的方式如下：

首先，对于SDR SDRAM，其每一个行/列选择的cell包含8bit Unit，而对于DDR SDRAM而言，其包含16bit Unit。

对于SDR而言，其在每个时钟的上升沿传输数据，在下降沿不进行操作。而对于DDR，其在每次上升沿和下降沿都传输数据，因此数据传输量为2倍。

![](w7p6.png)

如上图所示，其数据传输速率（每次要求读写放出的数据量）提高了一倍，但是核心频率和I/O频率没有变化。

![](w7p7.png)

就其内部而言，之前每周其可以放出1个数据，现在每周期可以放出2个数据。

注意，带宽的计算方法是使用等效频率计算的：200Mhz*64/8 = 1.6 Gb/s

这里的问题在于，我们需要一些数据，但是DDR给了我们更多的数据，它生成效率提高了2倍，但却不是真实的效率提高两倍，只是顺手干了别的活。

DDR2 SDRAM 相比较 DDR SDRAM 而言，其原理如下， 将I/O频率也提升了2倍，数据传输速率提升了2倍。

![](w7p8.png)

可以看到，其包含多条线链接到Memory Array，并同时从中读取数据进行I/O，因此，I/O频率提高带动了数据传输速率的提高。相当于提升了吞吐率（而从SDR到DDR可以看作提升了效率）

以下是从SDR到DDR到DDR2的每部分耗时标准：

![](w7p9.png)

![](w7p11.png)

![](w7p10.png)

可以看到，tRCD并没有显著减少，这取决于电容放电，而数据存取的效率和吞吐率都大幅提高了。但总的来说，处理器在30年性能提升了10000倍，而DRAM的性能才增长不到10倍，因此，CPU计算迫切依赖SRAM。

# SDRAM

## 基本元件读写

![](w7p12.png)

如上图所示，SRAM的基本存储单元由4个和2个组成，其中VDD代表电源，其值总是为1。当写入一个1的时候，WL（写入开关）为1，BL（写入数值）为1，而相对的另一端为0。因为WL为1，所以，M6通，M5通，M1通而M2不通，因此维持为1。同样的，M4通而M3不通，所以其维持为0，于是，当WL由1变为0的时候，内部电路维持稳定。

如下图所示。

![](w7p13.png)



当需要读出数据的时候，只用WL打开，M5 M6 连通，BL就为1了。

DRAM相比于SRAM而言，其构成简单，价格较低，但是读取受限于电容，速度慢。SRAM的集成度低，功耗高，价格高，但是速度快。

## 工作原理

**时空局部性**

根据经验性的结论：计算机程序在时间和空间上表现出局部性。其时间局部性指的是，最近被访问的存储器单元（指令或数据）很快还会被访问，比如循环。其空间局部性指的是，正在被访问的存储器单元附近的单元很快会被访问，比如数组。

Cache对于空间局部性的利用接住了Block，其和M交换的并非是以字为单位的，而是以数据块为单位的，这样如果下次程序需要使用空间附近数据，直接从cache中获取即可。

Cache对于时间局部性的利用在于其保存近期被频繁访问的主存单元的数据，方便下次使用。

**Cache 访问流程**

cache访问过程：CPU向Cache发出读请求，如果命中（Hit），那么从Cache返回读数据，如果不在，则定义为（Miss），接着向主存发送读请求，更新Cache并且返回读数据。

**Cache 组织结构**

cache的组织结构由 有效位（1bit） + 标签 + 数据 组成。

其中有效位表示当前数据是否有效，1为有效，0为无效。标签标识地址的头几位，其代表了存在该行数据的共同地址。数据则包含着地址的不同之处。如下图所示：

![](w7p14.png)

其中地址前两位被放在标签中，第三位用来标记表项1，最后一位用来表示存放在该行的哪一列。这样安排的原因是，一行有16个字节，而一位16进制包含相同的个数，所以用来区分一位。而倒数第二位则用行来区分，此外，标签用其共同的地址表示。

**Cache 的写策略**

类似于数据库，Cache命中和失效时都存在不同的写策略，较高速度的写策略不考虑安全性，只用将数据写入Cache，当Cache被替换时才送回内存，较高安全性的做法是直接写入Cache和主存，但是速度又太慢。

- “Cache命中”时的写策略
    -  写穿透（Write Through）：数据同时写入Cache和主存
    -  写返回（Write Back）：数据只写入Cache，仅当该数据块被替换时才将数据写回主存
- “ Cache失效”时的写策略
    -  写不分配（Write Non-Allocate）：直接将数据写入主存
    -  写分配（Write Allocate）：将该数据所在的块读入Cache后，再将数据写入Cache

**Cache 指标计算**

当Cache Hit的时候，统计Hit Rate，作为命中率。

当Cache Miss的时候，统计失效率。

当命中的时候，考虑命中时间 Hit Time，也就是从Cache将命中数据返回的时间，L1 Cache 约为1-3周期，L2 Cache约为5-20周期。

当失效的时候，考虑失效代价 Miss Penalty，一般为100-300个周期。

由此可以得到平均访存时间：

`AMAT = Hit Time + Miss Penalty × Miss Rate`

有趣的是，当命中率为97%时，平局访存时间为3+300*0.03=12，而当命中率为99%时，3+300×0.01=6，几乎提高了一倍的访问时间，但是命中率才提高了0.02就造成了此结果。

**Cache 失效的原因**

Cache 会因为第一次访问某一数据块而造成失效，称之为 Compulsory Miss，这不可避免。此外，当Cache无法保存程序访问所需的所有数据块，这称之为Capacity Miss容量失效，可以通过增加Cache容量实现。此外，还有一种是多个存储器位置映射到同一Cache位置造成的失效，称之为Conflict Miss，冲突失效，这个有不同的解决办法。

![](w7p15.png)

常见的Cache替换算法有：

- 随机（Random）
    - 硬件随机选择一个Cache块替换
- 轮转（Round-Robin）
    - 按照预先设定的顺序依次选择Cache块替换
- 最近最少使用（LRU）
    - 硬件记录访问历史信息
    - 选择距离现在最长时间未被访问的Cache块替换

对于Core i7而言，其高速缓存策略分为三级，L1包含分离数据和指令的32KB，8组路相连，4周期命中时间。L2包含统一的指令和数据，8组路相连，256KB，11周期命中时间。L3包含统一指令和数据，多核共享，8MB，16路相连，30-40枸杞命中。

# 存储容量

## 国际单位制前缀

`k M G T P E Z Y`

其分别对应1000的1、2、3、4、5、6、7、8次方，或者10的3、6、9...24次方。

`m u n p f a z y`

其分别对应1000的-1，-2...-8次方，或者10的-3，-6...-24次方。

计算机内部使用1024为底的数（CPU和内存），外部又使用国际单位制的数（硬盘），因此和国际单位制造成冲突。

因此，IEC提出的：

`Ki Mi Gi Ti Pi Ei Zi Yi` 来对应1024的1，2，3...8次方。

这样的话，现有的500GB硬盘对应466GiB，而内存则为512MiB。